In [2]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from mtcnn import MTCNN

In [3]:
def load_data(folder_path, num_images=None):
    images = []
    count = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(folder_path, filename))
            img = cv2.resize(img, (64, 64))  # Resize to a consistent size
            images.append(img)
            count += 1
            if num_images is not None and count >= num_images:
                break

    return np.array(images)

data_folder = '/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba'
X_train = load_data(data_folder, num_images=100000)
print(len(X_train)) 

100000


In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
labels = np.ones((X_train.shape[0], 1))  # Assuming all images contain faces

model.fit(X_train, labels, epochs=10, validation_split=0.2)


Epoch 1/10
2500/2500 [==============================] - 108s 43ms/step - loss: 1.3124e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
2500/2500 [==============================] - 97s 39ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
2500/2500 [==============================] - 93s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
2500/2500 [==============================] - 94s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
2500/2500 [==============================] - 93s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
2500/2500 [==============================] - 93s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
2500/2500 [==============================] - 94s 38ms/step - los

In [6]:
model.save("saved_models/sequential_model.h5")

/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
def predict_image(model, image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    return prediction[0][0]

new_image_path = '/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/test_photos/black.jpg'
# new_image_path = '/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba/201922.jpg'

model = load_model("saved_models/sequential_model.h5")
result = predict_image(model, new_image_path)
print(result)

if result >= 1:
    print("The image contains a cropped face.")
else:
    print("No cropped face detected in the image.")

1/1 [==============================] - 0s 147ms/step
0.75376767
No cropped face detected in the image.


In [22]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))  
    return img

def extract_patches(image, patch_size, stride):
    patches = []
    height, width, _ = image.shape

    for y in range(0, height - patch_size + 1, stride):
        for x in range(0, width - patch_size + 1, stride):
            patch = image[y:y+patch_size, x:x+patch_size, :]
            patches.append(patch)

    return patches

def predict_patches(model, patches):
    predictions = []
    for patch in patches:
        patch = np.expand_dims(patch, axis=0)
        prediction = model.predict(patch)
        predictions.append(prediction[0][0])
    return predictions

def detect_faces(predictions, threshold=0.5):
    average_score = np.mean(predictions)
    if average_score > threshold:
        return True
    else:
        return False
    
def blur_faces(image_path):
    detector = MTCNN()
    img = cv2.imread(image_path)

    faces = detector.detect_faces(img)

    for face in faces:
        x, y, width, height = face['box']
        roi = img[y:y + height, x:x + width]
        
        roi = cv2.GaussianBlur(roi, (99, 99), 30)

        img[y:y + roi.shape[0], x:x + roi.shape[1]] = roi

    # Display the result
    cv2.imshow("Blurred Faces", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


In [24]:
larger_image_path = '/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/test_photos/big_img_face.jpg'
larger_image = preprocess_image(larger_image_path)

patch_size = 64  
stride = 16  

model = load_model("saved_models/sequential_model.h5")

patches = extract_patches(larger_image, patch_size, stride)
predictions = predict_patches(model, patches)
result = detect_faces(predictions, threshold=0.999)

if result:
    blur_faces(larger_image_path)
    print("The larger image contains a face.")
else:
    print("No face detected in the larger image.")

1/1 [==============================] - 0s 53ms/step


: 